In [1]:
import warnings
warnings.filterwarnings('ignore')

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mixtape-Sessions/Machine-Learning/blob/main/Labs/RCT%20to%20Regression.ipynb)

ML enters the picture by providing an alternate way to generate $\hat{Y}_i$ and $\hat{D}_i$ when OLS is not the best tool for the job. The first two steps are really just prediction exercises, and in principle any supervised machine learning algorithm can step in here.

Steps 1 and 2 are prediction exercises--ML's wheelhouse. When OLS isn't the right tool for the job, we can replace OLS in those steps with machine learning:

1.   Predict $Y_{i}$ based on $X_{i}$ using ML and compute the residuals, $\tilde{Y}%
_{i}=Y_{i}-\hat{Y}_{i}^{ML}$, where $\hat{Y}_{i}^{ML}$ is the prediction from an ML algorithm
2.   Predict $D_{i}$ based on $X_{i}$ using ML and compute the residuals, $\tilde{D}%
_{i}=D_{i}-\hat{D}_{i}^{ML}$, where $\hat{D}_{i}^{ML}$ is the prediction from an ML algorithm

3. Regress $\tilde{Y}_{i}$ on $\tilde{D}_{i}$.

This is the basis for the two major methods we'll look at today: The first is "Post-Double Selection Lasso" (Belloni, Chernozhukov, Hansen). The second is "Double-Debiased Machine Learning" (Chernozhukov, Chetverikov, Demirer, Duflo, Hansen, Newey, Robins)

# Post Double Selection Lasso (PDS Lasso)

## Load useful packages:
pandas, numpy, linear_model (from sklearn), and KFold (from sklearn.model_selection)

Try it yourself first

### Cheat

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

## Read in data and have a look at it
We'll use the NLSY data

In [3]:
nlsy=pd.read_csv('https://github.com/brighamfrandsen/econ484/raw/master/data/nlsy97.csv?raw=true')
nlsy

,lnw_2016,educ,black,hispanic,other,exp,afqt,mom_educ,dad_educ,yhea_100_1997,...,_XPexp_13,_XPexp_14,_XPexp_16,_XPexp_17,_XPexp_18,_XPexp_19,_XPexp_20,_XPexp_21,_XPexp_22,_XPexp_23
0,4.076898,16,0,0,0,11,7.0724,12,12,3,...,0,0,0,0,0,0,0,0,0,0
1,3.294138,9,0,0,0,19,4.7481,9,10,2,...,0,0,0,0,0,1,0,0,0,0
2,2.830896,9,0,1,0,22,1.1987,12,9,3,...,0,0,0,0,0,0,0,0,1,0
3,4.306459,16,0,0,0,13,8.9321,16,18,2,...,1,0,0,0,0,0,0,0,0,0
4,5.991465,16,0,1,0,15,2.2618,16,16,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,1.833475,14,1,0,0,17,3.8179,15,12,2,...,0,0,0,1,0,0,0,0,0,0
1262,3.341985,9,0,1,0,20,3.3043,12,11,2,...,0,0,0,0,0,0,1,0,0,0
1263,-0.928125,10,1,0,0,19,1.0319,10,13,2,...,0,0,0,0,0,1,0,0,0,0
1264,3.702931,18,0,0,0,12,8.5093,16,19,2,...,0,0,0,0,0,0,0,0,0,0


## Define outcome, regressor of interest
y = lnw_2016

d = black

Try it yourself:

### Cheat

In [4]:
y=nlsy['lnw_2016']
d=nlsy[['black']]


## Simple Regression with no Controls
Regress y on d and print out coefficient
Try it yourself

In [5]:
# instantiate and fit a linear regression object

# print out regression coefficient


### Cheat

In [6]:
lm=linear_model.LinearRegression().fit(d,y)
print("Simple regression race gap: {:.3f}".format(lm.coef_[0]))

Simple regression race gap: -0.382


### ...
Is this the effect we're looking for?

Let's try a regression where we control for a few things: education (linearly), experience (linearly), and cognitive ability (afqt, linearly).

Try it yourself!

In [7]:
# define X, matrix of the d and the controls we want

# run regression

# print out coefficient


### Cheat

In [8]:
# define RHS, matrix of the d and the controls we want
RHS=nlsy[['black','educ','exp','afqt']]
# run regression
lm.fit(RHS,y)
# print out coefficient
print("Multiple regression-adjusted race gap: {:.3f}".format(lm.coef_[0]))



Multiple regression-adjusted race gap: -0.262



###...
How does it compare to the simple regression?

But who is to say the controls we included are sufficient? We have a whole host (hundreds!) of other potential controls, not to mention that perhaps the controls we did put in enter linearly. This is a job for ML!

To prep, let's define a matrix X with all of our potential controls:

In [9]:
X=nlsy.drop(columns=['lnw_2016','black'])
X

,educ,hispanic,other,exp,afqt,mom_educ,dad_educ,yhea_100_1997,yhea_2000_1997,yhea_2100_1997,...,_XPexp_13,_XPexp_14,_XPexp_16,_XPexp_17,_XPexp_18,_XPexp_19,_XPexp_20,_XPexp_21,_XPexp_22,_XPexp_23
0,16,0,0,11,7.0724,12,12,3,5,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,19,4.7481,9,10,2,5,10,...,0,0,0,0,0,1,0,0,0,0
2,9,1,0,22,1.1987,12,9,3,5,7,...,0,0,0,0,0,0,0,0,1,0
3,16,0,0,13,8.9321,16,18,2,5,10,...,1,0,0,0,0,0,0,0,0,0
4,16,1,0,15,2.2618,16,16,1,5,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,14,0,0,17,3.8179,15,12,2,5,11,...,0,0,0,1,0,0,0,0,0,0
1262,9,1,0,20,3.3043,12,11,2,6,0,...,0,0,0,0,0,0,1,0,0,0
1263,10,0,0,19,1.0319,10,13,2,5,3,...,0,0,0,0,0,1,0,0,0,0
1264,18,0,0,12,8.5093,16,19,2,6,0,...,0,0,0,0,0,0,0,0,0,0


## Post Double Selection Lasso

### Step 1: Lasso the outcome on X
Try it yourself. Don't forget to standardize Xs

#### Cheat

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xnames=X.columns
X = pd.DataFrame(scaler.fit_transform(X))
X.columns=Xnames
lassoy = linear_model.LassoCV(max_iter=1000).fit(X, y)

### Step 2: Lasso the treatment on X
Try it yourself

#### Cheat

In [11]:
lassod = linear_model.LassoCV(max_iter=100).fit(X, d)

### Step 3: Form the union of controls
Try it yourself

#### Cheat

In [12]:
Xunion=X.iloc[:,(lassod.coef_!=0) + (lassoy.coef_!=0)]
Xunion.head()

,educ,hispanic,other,afqt,mom_educ,yhea_2200_1997,youth_bothbio_01_1997,p4_001_1997,p5_102_1997,cv_bio_mom_age_child1_1997,...,_BGhp5_102__3,_BGhcvc_govc4,_BGhcvc_govd4,_BGhcvc_gove5,_BGhcvc_govo2,_BGhcvc_govp5,_BGhcvc_govp6,_XPexp_14,_XPexp_17,_XPexp_19
0,0.581418,-0.363840,-0.084616,0.599587,-0.542684,-0.990350,0.75546,-0.089551,-0.12282,1.353422,...,-0.056299,-0.028116,-0.028116,-0.028116,-0.056299,-0.028116,-0.039778,-0.330988,-0.32508,-0.273318
1,-1.833076,-0.363840,-0.084616,-0.198110,-1.792600,0.057713,0.75546,-1.066897,-0.12282,0.359263,...,-0.056299,-0.028116,-0.028116,-0.028116,-0.056299,-0.028116,-0.039778,-0.330988,-0.32508,3.658738
2,-1.833076,2.748464,-0.084616,-1.416260,-0.542684,1.180638,0.75546,0.887794,-0.12282,1.187729,...,-0.056299,-0.028116,-0.028116,-0.028116,-0.056299,-0.028116,-0.039778,-0.330988,-0.32508,-0.273318
3,0.581418,-0.363840,-0.084616,1.237833,1.123872,0.057713,0.75546,-0.089551,-0.12282,0.359263,...,-0.056299,-0.028116,-0.028116,-0.028116,-0.056299,-0.028116,-0.039778,-0.330988,-0.32508,-0.273318
4,0.581418,2.748464,-0.084616,-1.051405,1.123872,0.182483,0.75546,-0.089551,-0.12282,0.193570,...,-0.056299,-0.028116,-0.028116,-0.028116,-0.056299,-0.028116,-0.039778,-0.330988,-0.32508,-0.273318


### Concatenate treatment with union of controls and regress y on that and print out estimate
Try yourself

#### Cheat

In [13]:
rhs=pd.concat([d,Xunion],axis=1)
fullreg=linear_model.LinearRegression().fit(rhs,y)
print("PDS regression earnings race gap: {:.3f}".format(fullreg.coef_[0]))

PDS regression earnings race gap: -0.185


## Double-Debiased Machine Learning
For simplicity, we will first do it without sample splitting

### Step 1: Ridge outcome on Xs, get residuals
Try yourself

#### Cheat

In [14]:
ridgey = linear_model.RidgeCV().fit(X, y)
yresid=y-ridgey.predict(X)

### Step 2: Ridge treatment on Xs, get residuals
Try yourself

#### Cheat

In [15]:
ridged = linear_model.RidgeCV().fit(X, d)
dresid=d-ridged.predict(X)

### Step 3: Regress y resids on d resids and print out estimate
Try yourself

####Cheat

In [16]:
dmlreg=linear_model.LinearRegression().fit(dresid,yresid)
print("DML regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))

DML regression earnings race gap: -0.104


### The real thing: with sample splitting

In [17]:
# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)

# initialize columns for residuals
yresid = y*0
dresid = d*0

# Now loop through each fold
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index,:], d.iloc[test_index,:]

  # Do DML thing
  # Ridge y on training folds:
  ridgey.fit(X_train, y_train)

  # but get residuals in test set
  yresid.iloc[test_index]=y_test-ridgey.predict(X_test)

  #Ridge d on training folds
  ridged.fit(X_train, d_train)
  #but get residuals in test set
  dresid.iloc[test_index,:]=d_test-ridged.predict(X_test)


# Regress resids
dmlreg=linear_model.LinearRegression().fit(dresid,yresid)

print("DML regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))

DML regression earnings race gap: -0.197


You want standard errors, do you?

In [18]:
import statsmodels.api as sm
rhs = sm.add_constant(dresid)
model = sm.OLS(yresid, rhs)
results = model.fit(cov_type='HC3')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               lnw_2016   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     4.727
Date:                Thu, 21 Nov 2024   Prob (F-statistic):             0.0299
Time:                        16:51:07   Log-Likelihood:                -2250.2
No. Observations:                1266   AIC:                             4504.
Df Residuals:                    1264   BIC:                             4515.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0108      0.040     -0.267      0.7

## Now do DML using Random Forest!

### Cheat

In [ ]:
# import random forest
from sklearn.ensemble import RandomForestRegressor
# instantiate random forest objects
rfy=RandomForestRegressor(n_estimators=100)
rfd = rfy

# create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# apply the splits to our Xs
kf.get_n_splits(X)


# initialize columns for residuals
yresidrf = y*0
dresidrf = d*0

# Now loop through each fold
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  d_train, d_test = d.iloc[train_index,:], d.iloc[test_index,:]

  # Do DML thing
  # Ridge y on training folds:
  rfy.fit(X_train, y_train)

  # but get residuals in test set
  yresidrf.iloc[test_index]=y_test-rfy.predict(X_test)

  #Ridge d on training folds
  rfd.fit(X_train, d_train)
  #but get residuals in test set
  dresidrf.iloc[test_index,:]=d_test-rfd.predict(X_test).reshape(-1,1)


# Regress resids
dmlreg=linear_model.LinearRegression().fit(dresidrf,yresidrf)

print("DML (Random forest) regression earnings race gap: {:.3f}".format(dmlreg.coef_[0]))
rhs = sm.add_constant(dresidrf)
model = sm.OLS(yresidrf, rhs)
results = model.fit(cov_type='HC3')
print(results.summary())